<a href="https://colab.research.google.com/github/Pooja-Shanbhag/Predicting-Iris-Flower-Species/blob/master/Iris%20Species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#imports
from torch import Tensor
from torch.nn import Module
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torch.optim import SGD
from numpy import vstack
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from pandas import read_csv

In [8]:
class CSVDataset(Dataset):
    def __init__(self,path):
        dataframe = read_csv(path,header = None)
        self.X = dataframe.values[:,:-1]
        self.X = self.X.astype('float32')
        self.y = dataframe.values[:,-1]
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.reshape(len(self.y),1)
        
    def __getitem__(self,idx):
        return [self.X[idx],self.y[idx]]
    
    def __len__(self):
        return len(self.X)
    
    def split_data(self, n_train = 0.67):
        train_len = round(len(self.X)*n_train)
        test_len = len(self.X) - train_len
        return random_split(self, [train_len,test_len])

In [9]:
class MLP(Module):
    def __init__(self,inputs):
        super(MLP,self).__init__()
        self.hidden1 = Linear(inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity = 'relu')
        self.act1 = ReLU()
        
        self.hidden2 = Linear(10,8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
        self.act2 = ReLU()
        
        self.hidden3 = Linear(8,3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
        
    def forward(self,X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [10]:
def prepare_data(path):
    dataset = CSVDataset(path)
    train_dl, test_dl = dataset.split_data()
    train_dl = DataLoader(train_dl, batch_size = 32, shuffle = True)
    test_dl = DataLoader(test_dl, batch_size = 1024, shuffle = False)
    return train_dl, test_dl

In [12]:
def train_model(model,train_dl):
    criterion = CrossEntropyLoss()
    optim = SGD(model.parameters(), lr = 0.01, momentum = 0.9)
    for epoch in range(600):
        for i,(x,y) in enumerate(train_dl):
            optim.zero_grad()
            yhat = model(x)
            loss = criterion(yhat,y)
            loss.backward()
            optim.step()
            

In [13]:
def evaluate_model(model,test_dl):
    pred, actuals = list(), list()
    for i,(x,y) in enumerate(test_dl):
        yhat = model(x)
        
        yhat = yhat.detach().numpy()
        yhat = argmax(yhat,axis = 1)
        yhat = yhat.reshape(len(yhat),1)
        
        actual = y.numpy()
        actual = actual.reshape(len(actual),1)
        
        pred.append(yhat)
        actuals.append(y)
    pred, actuals = vstack(pred), vstack(actuals)
    acc = accuracy_score(actuals,pred)
    return acc
        
            